## U.S. Bureau of Labor Statistics - CPI Analysis
#### Eric Bottinelli

### 1. Retrieve data via BLS API v2

**Documentation**

- https://www.bls.gov/developers/api_python.htm
- https://data.bls.gov/cgi-bin/surveymost?cu
- https://data.bls.gov/dataQuery/find?fq=survey:[cu]&s=popularity:D&r=100&st=0
- https://www.bls.gov/cpi/tables/relative-importance/2023.htm

**Packages to install**

- Prettytable ('pip install prettytable')

**API Series ID**

Consumer Price Index for All Urban Consumers (CPI-U)
- *All items in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0
    - SA: CUSR0000SA0
- *All items less food and energy in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0L1E
    - SA: CUSR0000SA0L1E
- *Food in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAF1
    - SA: CUSR0000SAF1
- *Food at home in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAF11
    - SA: CUSR0000SAF11
- *Energy in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0E
    - SA: CUSR0000SA0E
- *Commodities less food and energy commodities in U.S. city average, all urban consumers*
    - NSA: CUUR0000SACL1E
    - SA: CUSR0000SACL1E4
- *Services less energy services in U.S. city average, all urban consumers*
    - NSA: CUUR0000SASLE
    - SA: CUSR0000SASLE
- *Shelter in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAH1
    - SA: CUSR0000SAH1
((https://www.bls.gov/cpi/factsheets/owners-equivalent-rent-and-rent.htm))

**Calculate special CPI**

Occasionally, a user wishes to estimate a price change that is not published by BLS. For instance, suppose a user would like a CPI series for ‘services less energy services and shelter’. This can be done by estimating a special index, in this case, ‘services less energy services and shelter’.
[BLS Doc](https://www.bls.gov/cpi/factsheets/constructing-special-cpis.htm)

If SEEB01 -> CUUR0000SEEB01

Cost weight is just a sum of all the items

If I add all the values to calculate the services less energy services and shelter, it becomes a lot of data. Explore different solution (e.g. remove goods from core CPI)

**Supercore CPI**

"Fed Chair Jerome Powell cited a specific category of inflation—inflation in core services other than housing—as being perhaps “the most important category for understanding the future evolution of core inflation.” The financial press has termed this category “supercore” inflation" ([FED of St. Louis](https://www.stlouisfed.org/on-the-economy/2024/may/measuring-inflation-headline-core-supercore-services))

In [1]:
import os
import requests
import json
import prettytable
import pandas as pd
from datetime import datetime

folder_name = 'CPI_Data'

In [129]:
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

current_date = datetime.now()
current_year = current_date.year
last_year = current_year - 1

series_names = {
    'CUUR0000SA0': 'NSA_All_Items',
    'CUSR0000SA0': 'SA_All_Items',
    'CUUR0000SA0L1E': 'NSA_All_Items_Less_Food_Energy',
    'CUSR0000SA0L1E': 'SA_All_Items_Less_Food_Energy',
    'CUUR0000SAF': 'NSA_Food',
    'CUSR0000SAF': 'SA_Food',
    'CUUR0000SAF11': 'NSA_Food_At_Home',
    'CUSR0000SAF11': 'SA_Food_At_Home',
    'CUUR0000SEFV': 'NSA_Food_Away_From_Home',
    'CUSR0000SEFV': 'SA_Food_Away_From_Home',
    'CUUR0000SA0E': 'NSA_Energy',
    'CUSR0000SA0E': 'SA_Energy',
    'CUUR0000SACL1E': 'NSA_Commodities_Less_Food_Energy_Commodities',
    'CUSR0000SACL1E4': 'SA_Commodities_Less_Food_Energy_Commodities',
    'CUUR0000SASLE': 'NSA_Services_Less_Energy_Services',
    'CUSR0000SASLE': 'SA_Services_Less_Energy_Services',
    'CUUR0000SAH1': 'NSA_Shelter',
    'CUSR0000SAH1': 'SA_Shelter',
}
series_ids = list(series_names.keys())

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": series_ids, "startyear": str(last_year), "endyear": str(current_year)})
response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(response.text)

all_data = []
for series in json_data['Results']['series']:
    rows = []
    for item in series['data']:
        footnotes = "".join([footnote['text'] + ',' for footnote in item['footnotes'] if footnote]).rstrip(',')
        if 'M01' <= item['period'] <= 'M12':
            rows.append([series_names[series['seriesID']], item['year'], item['period'], item['value'], footnotes])

    # Create dataframe for current series
    df = pd.DataFrame(rows, columns=["series id", "year", "period", "value", "footnotes"])
    all_data.append(df)

complete_data = pd.concat(all_data)

csv_path = os.path.join(folder_name, 'CPI_data.csv')
complete_data.to_csv(csv_path, index=False)

In [120]:
complete_data = pd.read_csv("CPI_Data/CPI_data.csv")

In [130]:
df = complete_data.copy()
df['date'] = pd.to_datetime(df['year'].astype(str) + df['period'].str.replace('M', ''), format='%Y%m')
df['series id'] = df['series id'].astype(str)  # Convert series id to string
df['value'] = pd.to_numeric(df['value'], errors='coerce')  # Ensure value is numeric
df['footnotes'] = df['footnotes'].astype(str)  # Convert footnotes to string
df.drop(['year', 'period', 'footnotes'], axis=1, inplace=True)
df.rename(columns={'series id': 'id'}, inplace=True)
df = df[['id', 'date', 'value']]

In [131]:
df['MoM_change'] = df.groupby('id')['value'].transform(lambda x: (x - x.shift(-1)) / x.shift(-1))
df['YoY_change'] = df.groupby('id')['value'].transform(lambda x: (x - x.shift(-12)) / x.shift(-12))

In [71]:
df.head(10)

,id,date,value,MoM_change,YoY_change
0,NSA_All_Items,2024-07-01,314.540,0.001162,0.028948
1,NSA_All_Items,2024-06-01,314.175,0.000338,0.029714
2,NSA_All_Items,2024-05-01,314.069,0.001662,0.032690
3,NSA_All_Items,2024-04-01,313.548,0.003893,0.033574
4,NSA_All_Items,2024-03-01,312.332,0.006464,0.034774
5,NSA_All_Items,2024-02-01,310.326,0.006190,0.031532
6,NSA_All_Items,2024-01-01,308.417,0.005448,0.030909
7,NSA_All_Items,2023-12-01,306.746,-0.000993,NaN
8,NSA_All_Items,2023-11-01,307.051,-0.002015,NaN
9,NSA_All_Items,2023-10-01,307.671,-0.000383,NaN


In [145]:
food_weight = 0.136
food = df.loc[df['id'] == 'SA_Food', ['MoM_change','YoY_change']] * food_weight
energy_weight = 0.067
energy = df.loc[df['id'] == 'SA_Energy', ['MoM_change','YoY_change']] * energy_weight
food_energy = food + energy
food_energy['id'] = 'SA_Food_Energy'
food_energy['date'] = df.loc[df['id'] == 'SA_Food', 'date']
food_energy['value'] = 0
food_energy = food_energy[['id', 'date', 'value', 'MoM_change', 'YoY_change']]
df = pd.concat([df, food_energy])

food = df.loc[df['id'] == 'NSA_Food', ['MoM_change','YoY_change']] * food_weight
energy = df.loc[df['id'] == 'NSA_Energy', ['MoM_change','YoY_change']] * energy_weight
food_energy = food + energy
food_energy['id'] = 'NSA_Food_Energy'
food_energy['date'] = df.loc[df['id'] == 'NSA_Food', 'date']
food_energy['value'] = 0
food_energy = food_energy[['id', 'date', 'value', 'MoM_change', 'YoY_change']]
df = pd.concat([df, food_energy])

In [142]:
food_weight + energy_weight

0.203

In [146]:
category_map = {
    'All_Items': (0, 'Headline', '', ''),
    'Food_Energy': (1, 'Food + Energy', '', ''),
    'Food': (2, '', 'Food', ''),
    'Food_At_Home': (3, '', '', 'At home'),
    'Food_Away_From_Home': (4, '', '', 'Away Home'),
    'Energy': (5, '', 'Energy', ''),
    'All_Items_Less_Food_Energy': (6, 'Core', '', ''),
    'Commodities_Less_Food_Energy_Commodities': (7, '', 'Commodities', ''),
    'Services_Less_Energy_Services': (8, '', 'Services', ''),
    'Shelter': (9, '', '', 'Shelter'),
}

weight_map = {
    'All_Items': '100%',
    'Food_Energy': f'{round((food_weight + energy_weight)*100,1)}%',
    'Food': '13.6%',
    'Food_At_Home': '8.2%',
    'Food_Away_From_Home': '5.4%',
    'Energy': '6.7%',
    'All_Items_Less_Food_Energy': '79.8%',
    'Commodities_Less_Food_Energy_Commodities': '18.9%',
    'Services_Less_Energy_Services': '60.9%',
    'Shelter': '36.2%'
}

ordered_categories = ['Headline', 'Food + Energy', 'Core']
ordered_sub_categories_1 = ['Food', 'Energy', 'Commodities', 'Services']
ordered_sub_categories_2 = ['At home', 'Away Home', 'Shelter', 'Supercore']

for i in range(4):
    if i in [0, 1]:
        data = df[df['id'].str.startswith('NSA_')].copy()  # Ensure it's a copy to avoid SettingWithCopyWarning
        prefix_length = 4
    else:
        data = df[df['id'].str.startswith('SA_')].copy()  # Ensure it's a copy to avoid SettingWithCopyWarning
        prefix_length = 3

    # Use .loc to safely modify the 'id' column
    data.loc[:, 'id'] = data['id'].str[prefix_length:]

    # Verify all IDs are in category_map
    missing_ids = set(data['id']) - set(category_map.keys())
    if missing_ids:
        raise ValueError(f"Missing IDs in category_map: {missing_ids}")

    # Safely setting new column values
    data.loc[:, 'Month-Year'] = data['date'].dt.strftime('%b-%y')
    order_cat = data['id'].apply(lambda x: category_map.get(x, (None, None, None, None)))
    data.loc[:, 'Order'] = [item[0] for item in order_cat]
    data.loc[:, 'Category'] = [item[1] for item in order_cat]
    data.loc[:, 'Sub Category 1'] = [item[2] for item in order_cat]
    data.loc[:, 'Sub Category 2'] = [item[3] for item in order_cat]
    data.loc[:, 'Weight'] = data['id'].map(weight_map).fillna('Unknown')

    if i in [0, 2]:
        value_column = 'MoM_change'
    else:
        value_column = 'YoY_change'

    data_pivot = data.pivot_table(
        index=['Order', 'Category', 'Sub Category 1', 'Sub Category 2', 'Weight'],
        columns='Month-Year',
        values=value_column,
        aggfunc='first'
    )

    data_pivot = data_pivot[sorted(data_pivot.columns, key=lambda x: pd.to_datetime(x, format='%b-%y'), reverse=True)]
    data_pivot.columns.name = None
    data_pivot.reset_index(inplace=True)
    data_pivot.sort_values(by='Order', inplace=True)
    data_pivot.drop(columns=['Order'], inplace=True)

    if i == 0:
        NSA_MoM_df = data_pivot
    elif i == 1:
        NSA_YoY_df = data_pivot
    elif i == 2:
        SA_MoM_df = data_pivot
    else:
        SA_YoY_df = data_pivot

In [147]:
csv_path = os.path.join(folder_name, 'NSA_MoM_CPI_data.csv')
NSA_MoM_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'NSA_YoY_CPI_data.csv')
NSA_YoY_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'SA_MoM_CPI_data.csv')
SA_MoM_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'SA_YoY_CPI_data.csv')
SA_YoY_df.to_csv(csv_path, index=False)